In [1]:

import src
# TODO: Delete after development
import importlib
importlib.reload(src)

from src import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
authors = []
for author_name in FileUtils.read_authors(Configuration.SELECTED_AUTHORS_FILEPATH):
    author = Author(name=author_name)
    author.read_book_collection(Configuration.ALL_BOOK_DATA_DIR)
    author.read_generated_texts(Configuration.MODELS_DATA_DIR)
    authors.append(author)

In [3]:
analysis = Analysis(Configuration.SIZE_ANALYSIS)
analysis_data = analysis.analyze(authors)

In [6]:
for author_name, collections_data in analysis_data.items():
        AnalysisVisualization().visualize(collections_data, f"{author_name} analysis")

In [5]:
# from functionwords import FunctionWords

# fw = FunctionWords(function_words_list="english")
# fw.transform(authors_gpt_3_5_turbo["Zane Grey"][0]["response"])
# fw
# # to list all function words given `function_words_list`
# # returns a list
# # fw.get_feature_names()

# std error of mean
# signifiacne test
# 2-sample t-test